In [187]:
import urllib3
from urllib.parse import urlencode
import pandas as pd

In [188]:
http = urllib3.PoolManager()

We scrap all the ids of each public consultation

In [189]:

initiative_summary_page_list = []

url = 'https://ec.europa.eu/info/law/better-regulation/brpapi/searchInitiatives'



def scrap_initiative_id(page_id):
    print(f"Scraping page {page_id}", end="\r")
    params = {
        "frontEndStage": "OPC_LAUNCHED",
        "size": 100,
        "page": page_id
        }
    encoded_params = urlencode(params)
    full_url = f'{url}?{encoded_params}'

    response = http.request("GET", full_url)
    json = response.json()
    return json

first_page_json = scrap_initiative_id(0)
initiative_summary_page_list.append(first_page_json)

elements_count = first_page_json['page']['totalElements']
total_pages = first_page_json['page']['totalPages']
print(f"number of elements : {elements_count}")
print(f"number of pages : {total_pages}")


for i_page in range(1, total_pages):
    initiative_summary_page_list.append(scrap_initiative_id(i_page))

number of elements : 619
number of pages : 7


In [190]:
initiative_summary_list = []
for page in initiative_summary_page_list:
    for initiative_summary in page['_embedded']['initiativeResultDtoes']:
        initiative_summary_list.append(initiative_summary)
    

In [191]:
set([x['currentStatuses'][0]['receivingFeedbackStatus'] for x in initiative_summary_list])

{'CLOSED', 'OPEN', 'UPCOMING'}

In [193]:
out_list = []

i=0
for initiative_summary in initiative_summary_list:
    print(f'scraping id {i} of {len(initiative_summary_list)}', end='\r')
    i+=1

    initiative_id = int(initiative_summary["id"])
    short_title = initiative_summary['shortTitle']
    contact_mail = ''
    receiving_feedback_status = initiative_summary['currentStatuses'][0]['receivingFeedbackStatus']
    feedback_start_date = initiative_summary['currentStatuses'][0]['feedbackStartDate']
    feedback_end_date = initiative_summary['currentStatuses'][0]['feedbackEndDate']
    page_url = f'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/{initiative_id}'

    if receiving_feedback_status in {'CLOSED', 'OPEN'}:
        url = f'https://ec.europa.eu/info/law/better-regulation/brpapi/publicConsultation/{int(initiative_id)}'
        response = http.request("GET", url)
        if (response.status != 200):
            print(f'request {url} didn\'t work')
            break
        json = response.json()
        contact_mail = json['surveyContactMail']


    out_list.append([
        short_title,
        initiative_id,
        contact_mail, 
        receiving_feedback_status,
        feedback_start_date,
        feedback_end_date,
        page_url
    ])
    

In [198]:
df = pd.DataFrame(out_list, columns=['short_title',
        'initiative_id',
        'contact_mail', 
        'receiving_feedback_status',
        'feedback_start_date',
        'feedback_end_date',
        'page_url'])

In [200]:
df.to_csv('public_consultations_contacts.csv')